In [1]:
import theano
import lasagne
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
import nltk.data

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from nolearn.lasagne import NeuralNet
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import BatchIterator
from lasagne.layers import InputLayer, Conv2DLayer, DropoutLayer,\
    MaxPool2DLayer, DenseLayer
from lasagne.nonlinearities import softmax, sigmoid
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.4/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('data/imdb.csv')

In [3]:
def agrupar_pelis(x):
    if x >=7.5:
        return 2
    elif x<7.5 and x>=6:
        return 1
    else:
        return 0

df['imdb_score'] = df['imdb_score'].apply(agrupar_pelis)

In [4]:
ratings = np.sort(df['imdb_score'].unique())

In [5]:
len(df[df.imdb_score == 0]), len(df[df.imdb_score == 1]),len(df[df.imdb_score == 2])

(1454, 2702, 887)

In [6]:
#df['earning_rate'] = df.apply(lambda r: r.gross / r.budget, axis=1)

In [7]:
a = df.corr()

In [8]:
import matplotlib.pyplot as plt

df_corr = df[a.columns.tolist()]

plt.matshow(df_corr.corr())
plt.xticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist(), rotation="vertical")
plt.yticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist())
plt.colorbar()
plt.show()

/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [9]:
ac1 = df['actor_1_name'].unique()
ac2 = df['actor_2_name'].unique()
ac3 = df['actor_3_name'].unique()

In [10]:
total = np.concatenate((ac1, ac2, ac3))

In [11]:
total = dict(enumerate(set(total)))

In [12]:
def factorizar(row):
    for x in total:
        if total[x] == row:
            return int(x)

In [13]:
df['actor_1_name'] = df['actor_1_name'].apply(lambda row: factorizar(row))
df['actor_2_name'] = df['actor_2_name'].apply(lambda row: factorizar(row))
df['actor_3_name'] = df['actor_3_name'].apply(lambda row: factorizar(row))

In [14]:
## Para obtener los indices de los paises y los lenguajes de las peliculas
idx_language, label_language = pd.factorize(df['language'])
idx_country, label_country = pd.factorize(df['country'])

## Para obtener los indices de los actores
#idx_actor1, label_actor1 = pd.factorize(df['actor_1_name'])
#idx_actor2, label_actor2 = pd.factorize(df['actor_2_name'])
#idx_actor3, label_actor3 = pd.factorize(df['actor_3_name'])

## Para obtener el indice del director
idx_director, label_director = pd.factorize(df['director_name'])

#Para obtener los indices del titulo de las peliculas y el content rating
idx_movie, label_movie = pd.factorize(df['movie_title'])
idx_content, label_content = pd.factorize(df['content_rating'])

In [15]:
## Separa los generos en columnas
df = pd.concat([df, df.genres.str.get_dummies(sep='|')], axis=1)

In [16]:
df_separados = pd.DataFrame()
df_separados = pd.concat([df_separados, df.plot_keywords.str.replace('|',' ')],axis=1)

nltk.data.path = ['nltk_data/']

stopw = set(stopwords.words('english')) 
corpus = []
for e in df_separados.values:
    if type(e[0]) is str:
        corpus.append(e[0])
corpus1 = " ".join(corpus)
corpus1 = corpus1.split(" ")

In [17]:
corpus1 = list(filter(lambda x: not x[0] in stopw, corpus1))
corpus1 = set(corpus1)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vocabulary = corpus1
vect = TfidfVectorizer(sublinear_tf=True, max_df=2, analyzer='word', 
           stop_words=stopw, vocabulary=vocabulary)
vect.fit(corpus)
#print(dict(zip(vect.get_feature_names(), vect.idf_)))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=2, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'above', 'o', 'wouldn', 'so', 'those', 'now', 'why', 'when', 'yourself', 'few', 'with', 'which', 'their', 'shan', 's', 'its', 'some', 'did', 'because', 'how', 'isn', 'itself', 'has', 'very', 'about', 'his', 'have', 'below', 'once', 'and', 'nor', 'it', 'we', 'against', 'myself', 'to', 'wh...s', 'all', 'of', 'will', 'wasn', 'had', 'each', 'mustn', 'her', 'mightn', 'are', 'into', 've', 'my'},
        strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary={'pimp', 'phobia', 'usa', 'liberace', 'crematorium', 'factory', 'rune', 'focus', 'homecoming', 'paradise', 'laboratory', 'coven', 'ronin', 'banana', 'resurrection', 'en

In [19]:
def transformar_keywords(k):
    if not type(k) is str:
        return 0  
    keywords = k.replace('|'," ")
    doc_tfidf = vect.transform([k])
    t = doc_tfidf.toarray().sum()
    return t

In [20]:
df['plot_keywords'] = df['plot_keywords'].apply(lambda row: transformar_keywords(row)) 

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [21]:
df['language'] = idx_language
df['country'] = idx_country
#df['actor_1_name'] = idx_actor1
#df['actor_2_name'] = idx_actor2
#df['actor_3_name'] = idx_actor3
df['director_name'] = idx_director
df['movie_title'] = idx_movie
df['content_rating'] = idx_content

In [22]:
df.fillna(value=-1, inplace=True)

In [23]:
y = df['imdb_score']
df.drop('imdb_score', axis=1, inplace=True)

### Columnas Dropeadas

In [24]:
#df.columns

In [25]:
#df.drop('actor_1_facebook_likes', axis=1, inplace=True)
#df.drop('actor_2_facebook_likes', axis=1, inplace=True)
#df.drop('actor_3_facebook_likes', axis=1, inplace=True)
#df.drop('director_facebook_likes', axis=1, inplace=True)

#df.drop('director_name', axis=1, inplace=True)
#df.drop('actor_1_name', axis=1, inplace=True)
#df.drop('actor_2_name', axis=1, inplace=True)
#df.drop('actor_3_name', axis=1, inplace=True)

#df.drop('cast_total_facebook_likes', axis=1, inplace=True)
#df.drop('movie_facebook_likes', axis=1, inplace=True)
#df.drop('budget', axis=1, inplace=True)
#df.drop('gross', axis=1, inplace=True)
#df.drop('cast_total_facebook_likes', axis=1, inplace=True)
#df.drop('plot_keywords', axis=1, inplace=True)
df.drop('genres', axis=1, inplace=True)
df.drop('movie_imdb_link', axis=1, inplace=True)
df.drop('color', axis=1, inplace=True)
#df.drop('num_user_for_reviews', axis=1, inplace=True)
#df.drop('movie_title', axis=1, inplace=True)
#df.drop('content_rating', axis=1, inplace=True)
#df.drop('duration', axis=1, inplace=True)

In [26]:
xs = x = df.values
ys = y

## Matriz de Correlación Full

In [27]:
a = df.corr()

df_corr = df[a.columns.tolist()]

plt.matshow(df_corr.corr())
plt.xticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist(), rotation="vertical")
plt.yticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist())
plt.colorbar()
plt.show()

/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


# PCA

In [28]:
from sklearn.decomposition import RandomizedPCA # using randomized Singular Value Decomposition 
Xp = RandomizedPCA(n_components=35, random_state=1)
Xp = Xp.fit_transform(xs)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


In [29]:
#Xp.explained_variance_ratio_.cumsum()

In [30]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(xs, ys,random_state=1)

In [32]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics


def plot_matrix(clf, X_test, y_test):
    plt.clf()
    plt.imshow(confusion_matrix(clf.predict(X_test), y_test),
               interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    plt.show()

In [33]:
clf = ExtraTreesClassifier(n_estimators=300,
                           max_features=0.2, 
                           n_jobs=2,
                           max_depth=None,
                           min_samples_split=1,
                           random_state=1).fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))
print("Score over Testing Data {}".format(clf.score(X_test, y_test)))
print("Score over Training Data {}".format(clf.score(X_train, y_train)))
plot_matrix(clf, X_test, y_test)

             precision    recall  f1-score   support

          0       0.76      0.62      0.68       379
          1       0.70      0.85      0.77       658
          2       0.82      0.58      0.68       224

avg / total       0.74      0.73      0.73      1261

Score over Testing Data 0.7319587628865979
Score over Training Data 1.0


/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [34]:
#Se guardan scores y labels verdaderos del ExtraTreesClassifier
y_true_tree = y_test
scores_tree = clf.predict(X_test)

In [35]:
importances = clf.feature_importances_
cols = len(df.columns)

text = list(map(lambda i: df.columns[i], range(cols)))
plt.figure(figsize=(20,cols))
print(importances[::-1].shape)
plt.bar(range(cols),height=importances,  width=1.)
plt.xticks(np.arange(0.5, cols, 1.), text, rotation=90)
plt.xlim((0, cols))
plt.show()

(50,)


In [36]:
confusion_matrix(y_test, clf.predict(X_test))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


array([[235, 143,   1],
       [ 71, 559,  28],
       [  3,  92, 129]])

## Dummy

In [37]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(random_state=0).fit(X_train, y_train)
#print clf.score(X_test, y_test)
plot_matrix(clf, X_test, y_test)
clf.score(X_test, y_test)
print(classification_report(y_test, clf.predict(X_test)))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


             precision    recall  f1-score   support

          0       0.30      0.28      0.29       379
          1       0.52      0.53      0.52       658
          2       0.17      0.18      0.18       224

avg / total       0.39      0.39      0.39      1261



In [38]:
confusion_matrix(y_test, clf.predict(X_test))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


array([[106, 212,  61],
       [175, 346, 137],
       [ 76, 107,  41]])

## SVM

In [39]:
from sklearn.svm import SVC

sv = SVC(kernel='rbf', cache_size=1000)
sv.fit(X_train, y_train)

print(classification_report(y_test, sv.predict(X_test)))
print(sv.score(X_test, y_test))
plot_matrix(sv, X_test, y_test)



             precision    recall  f1-score   support

          0       1.00      0.03      0.06       379
          1       0.53      1.00      0.69       658
          2       1.00      0.03      0.05       224

avg / total       0.75      0.54      0.39      1261

0.536082474227


/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [40]:
confusion_matrix(y_test, sv.predict(X_test))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


array([[ 12, 367,   0],
       [  0, 658,   0],
       [  0, 218,   6]])

In [41]:
y_true_svm = y_test
scores_svm = sv.predict(X_test)

# Red Neuronal

In [42]:
from lasagne.nonlinearities import linear, tanh, rectify
from sklearn.metrics import classification_report, confusion_matrix

In [43]:
xs = x
ys = np.array(y)

In [44]:
std_x = MinMaxScaler([-1, 1])
xs = std_x.fit_transform(np.array(xs))
std_y = MinMaxScaler([0, 2])
ys = std_y.fit_transform(np.array(ys))

X_train, X_test, y_train, y_test = train_test_split(xs, ys,random_state=1)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [45]:
layers_0 = [
                (InputLayer, {'shape': (None, 50)}),
                (DenseLayer, {'num_units': 512}),
                (DropoutLayer, {'p': 0.5}),
                (DenseLayer, {'num_units': 512}),
                (DropoutLayer, {'p': 0.5}),
                (DenseLayer, {'num_units': 3, 'nonlinearity': softmax}),
        ]

In [46]:
class AdjustVariable(object):
    """
    Used to decreases linearly the learning rate with the number of epochs,
    while we the momentum increase.
    """
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = np.float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [47]:
def create_network(npochs=50, batch_s=10000):
    return NeuralNet(
        layers=layers_0,
        update=nesterov_momentum,
        update_learning_rate=theano.shared(np.float32(0.009)),
        update_momentum=theano.shared(np.float32(0.9)),

        regression=False,
        batch_iterator_train=BatchIterator(batch_size=batch_s),
        on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.09, stop=0.009),
            AdjustVariable('update_momentum', start=0.9, stop=0.9999)
        ],
        max_epochs=npochs,
        verbose=1)

#net0 = create_network(10)

In [49]:
net0 = create_network(60)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int32)

net0.fit(X_train, y_train)

In [52]:
plot_loss_net(net0)

In [51]:
#%matplotlib inline

def plot_loss_net(net0):
    plt.clf()
    plt.figure(figsize=(15,5))

    train_loss = np.array([i["train_loss"] for i in net0.train_history_])
    valid_loss = np.array([i["valid_loss"] for i in net0.train_history_])
    plt.plot(train_loss, '--b', linewidth=2, label="{} train".format("net0"))
    plt.plot(valid_loss, '-b', linewidth=2, label="{} valid".format("net0"))

    plt.grid()
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.yscale("log")
    plt.show()

In [53]:
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int32)

print(classification_report(y_test, net0.predict(X_test)))
plot_matrix(net0, X_test, y_test)

             precision    recall  f1-score   support

          0       0.64      0.58      0.61       379
          1       0.66      0.76      0.71       658
          2       0.69      0.48      0.56       224

avg / total       0.66      0.66      0.65      1261



/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)
/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


# <span style="color:red"> VER ESTO!! NO ESTA TOMANDO LOS ELEMENTOS DE LAS CLASES 0 Y 2</span>.

In [54]:
confusion_matrix(y_test, net0.predict(X_test))

/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


array([[220, 154,   5],
       [112, 502,  44],
       [ 10, 107, 107]])

### Curva ROC 

In [55]:

def get_color_plt(i):
    color = ""
    if i == 0:
        color = 'darkorange'
    elif i == 1:
        color = 'blue'
    elif i== 2: 
        color = 'green'
    elif i == 3:
        color = 'red'
    return color

# Ploteo para cada una de las clases
def plotear_grafico_roc(lista_fpr,lista_tpr,lista_nombres):
    plt.figure()
    lw = 2
    for i in range(0,len(lista_fpr)):
        # Compute ROC curve and ROC area for each class
        roc_auc = dict()
        print (lista_fpr[i])
        roc_auc[0] = auc(lista_fpr[i], lista_tpr[i])
        color = get_color_plt(i)
        plt.plot(lista_fpr[i], lista_tpr[i], color=color,
                 lw=lw,
                 label='ROC curve (%s, area = %0.2f)' % (lista_nombres[i],roc_auc[0]))
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()


In [63]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

y_true_net = y_test
scores_net = net0.predict(X_test)
fpr_net, tpr_net, thresholds = roc_curve(y_true_net, scores_net, pos_label=2)
fpr_svm, tpr_svm, thresholds = roc_curve(y_true_svm, scores_svm, pos_label=2)
fpr_tree, tpr_tree, thresholds = roc_curve(y_true_tree, scores_tree, pos_label=2)


plotear_grafico_roc([fpr_net,fpr_svm,fpr_tree],[tpr_net,tpr_svm,tpr_tree],['ANN','SVC','TreeClassifier'])



[ 0.          0.04725169  0.67984571  1.        ]
[ 0.          0.98842816  1.        ]
[ 0.          0.02796528  0.70491803  1.        ]
